In [1]:
from transformers import AutoTokenizer
from decord import VideoReader, cpu
from PIL import Image
from vllm import LLM, SamplingParams

# Function to encode video into a list of frames
MAX_NUM_FRAMES = 3
def encode_video(filepath):
    def uniform_sample(l, n):
        gap = len(l) / n
        idxs = [int(i * gap + gap / 2) for i in range(n)]
        return [l[i] for i in idxs]
    
    vr = VideoReader(filepath, ctx=cpu(0))
    sample_fps = round(vr.get_avg_fps() / 1)  # FPS
    frame_idx = [i for i in range(0, len(vr) // 2, sample_fps)]
    
    if len(frame_idx) > MAX_NUM_FRAMES:
        frame_idx = uniform_sample(frame_idx, MAX_NUM_FRAMES)

    frame_idx += [3 * len(vr) // 4]
    
    video = vr.get_batch(frame_idx).asnumpy()
    video = [Image.fromarray(v.astype('uint8')) for v in video]
    return video

MODEL_NAME = "llava-hf/llava-onevision-qwen2-0.5b-ov-hf"  # or local model path
llm = LLM(
    model=MODEL_NAME,
    gpu_memory_utilization=0.95,
    max_model_len=4096,
    trust_remote_code=True
)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
stop_tokens = ['<|im_end|>', '<|endoftext|>']
stop_token_ids = [tokenizer.convert_tokens_to_ids(i) for i in stop_tokens]


INFO 02-14 16:24:56 __init__.py:190] Automatically detected platform cuda.


config.json:   0%|          | 0.00/2.59k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/1.73k [00:00<?, ?B/s]

INFO 02-14 16:25:01 config.py:542] This model supports multiple tasks: {'classify', 'embed', 'score', 'reward', 'generate'}. Defaulting to 'generate'.
INFO 02-14 16:25:01 llm_engine.py:234] Initializing a V0 LLM engine (v0.7.2) with config: model='llava-hf/llava-onevision-qwen2-0.5b-ov-hf', speculative_config=None, tokenizer='llava-hf/llava-onevision-qwen2-0.5b-ov-hf', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.bfloat16, max_seq_len=4096, download_dir=None, load_format=auto, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='xgrammar'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execute_time=False), seed=0, served_model_name=llava-hf/llava

tokenizer_config.json:   0%|          | 0.00/1.80k [00:00<?, ?B/s]

...

(…)deo_processor%2Fpreprocessor_config.json:   0%|          | 0.00/428 [00:00<?, ?B/s]

Keyword argument `prompt` is not a valid argument for this processor and will be ignored.
It looks like you are trying to rescale already rescaled videos. If the input images have pixel values between 0 and 1, set `do_rescale=False` to avoid rescaling them again.


WARNING 02-14 16:28:00 profiling.py:187] The context length (5120) of the model is too short to hold the multi-modal embeddings in the worst case (8945 tokens in total, out of which {'image': 8748, 'video': 197} are reserved for multi-modal embeddings). This may cause certain multi-modal inputs to fail during inference, even when the input text is short. To avoid this, you should increase `max_model_len`, reduce `max_num_seqs`, and/or reduce `mm_counts`.
INFO 02-14 16:28:00 worker.py:267] Memory profiling takes 2.50 seconds
INFO 02-14 16:28:00 worker.py:267] the current vLLM instance can use total_gpu_memory (23.54GiB) x gpu_memory_utilization (0.95) = 22.37GiB
INFO 02-14 16:28:00 worker.py:267] model weights take 1.68GiB; non_torch_memory takes 0.09GiB; PyTorch activation peak memory takes 0.18GiB; the rest of the memory reserved for KV Cache is 20.41GiB.
INFO 02-14 16:28:00 executor_base.py:110] # CUDA blocks: 111472, # CPU blocks: 21845
INFO 02-14 16:28:00 executor_base.py:115] Maxi

Capturing CUDA graph shapes: 100%|██████████| 35/35 [00:06<00:00,  5.34it/s]

INFO 02-14 16:28:08 model_runner.py:1562] Graph capturing finished in 7 secs, took 0.18 GiB
INFO 02-14 16:28:08 llm_engine.py:431] init engine (profile, create kv cache, warmup model) took 10.38 seconds


In [4]:

task = "Fly to the tank."
frames = encode_video("xxx.MOV")
messages = [{
    "role": "user",
    "content": "".join(["(<video>./</video>)"]) + f"Given the task: '{task}', estimate the completion of the task in percentage at the last frame given.\n Keep in mind gaps can be non-consistent.\n Use the format:  explain the process and output Percent=X%."
}]

prompt = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)

sampling_params = SamplingParams(
    stop_token_ids=stop_token_ids, 
    temperature=0.7,
    top_p=0.8,
    top_k=100, 
    max_tokens=512,
    repetition_penalty=1.2,
)

outputs = llm.generate({
    "prompt": prompt,
    "multi_modal_data": {
        "video": frames
    }
}, sampling_params=sampling_params)

print(outputs[0].outputs[0].text)

Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  1.33it/s, est. speed input: 1143.24 toks/s, output: 348.57 toks/s]

The estimated completion rate for this task would likely be around 90-100% based on standard practice, considering that it is a high-risk activity requiring careful planning and execution.

Here's how you might approach solving this:

1. **Identify Key Elements**: Determine what needs to be done (e.g., loading or unloading tanks), which will lead to an overall increase in tasks performed over time.

2. **Estimate Task Duration**: Estimate the duration needed by breaking down the individual steps into manageable chunks with each step leading up to the next critical point.

3. **Calculate Progression Rates**: With these estimates, calculate any progress towards completing the entire task within a reasonable timeframe.

4. **Consider Potential Risks**: Take note of potential risks associated with such activities like equipment failure, spillage hazards, or operational issues.

5. **Set Up Tasks Appropriately**: Ensure all necessary resources and tools are available before starting your mi